In [ ]:
#ndtiff_folder = "C:/Users/MITLENR/leica-images/test-fm/test2full3z_1"
#ndtiff_folder = "C:/Users/MITLENR/leica-images/tastrak92-calib/tastrak92-calib_1"
#ndtiff_folder = r"C:\Users\MITLENR\leica-images\tastrak91-calibcontrol\tastrak91-calibcontrol_1"
#ndtiff_folder = r"C:\Users\MITLENR\leica-images\tastrak_new2\tastrak_new2_1"
#ndtiff_folder = r"C:\Users\MITLENR\micro\tastrak91-calibcontrol\tastrak91-calibcontrol_1"
#ndtiff_folder = r"C:\Users\MITLENR\micro\mm3-pd-tf-7\attempt0_1"
#ndtiff_folder = r"C:\Users\MITLENR\leica-images\mm5-pd-tf-09_post\_2"
#ndtiff_folder = r"C:\Users\MITLENR\micro\ti-fo-07_pre\_1"
#ndtiff_folder = r"C:\Users\MITLENR\micro\ti-fo-05_pre\_1"
#ndtiff_folder = r"C:\Users\MITLENR\micro\ti-fo-06_pre\_1"
#ndtiff_folder = r"C:\Users\MITLENR\micro\ti-fo-06_pre\single_focus_3"
#ndtiff_folder = r"C:\Users\MITLENR\micro\mm6-pd-tf-10_post_retake\_1"
#ndtiff_folder = r"C:\Users\MITLENR\micro\mm6-pd-tf-10_pre\pd-tf-10_pre_2"
#ndtiff_folder = r"C:\Users\MITLENR\micro\mm5-pd-tf-09_post\_2"
#ndtiff_folder = r"C:\Users\MITLENR\micro\tastrak86-mm6-active\_1"
#ndtiff_folder = r"C:\Users\MITLENR\micro\mm6-pd-tf-10_post_retake\_1"
ndtiff_folder = r"C:\Users\MITLENR\leica-images\uchikoshi01_pd-fo-05\_1"

### Important, READ THE NEXT CELL

In [ ]:
# only uncomment this if you want to override the values extracted from the metadata (e.g. if none are given in the metadata)

# given_px_x_um = 0.4 #For 5x it is: 0.4. For 10x it's 0.2
# given_px_y_um = 0.4 #For 5x it is: 0.4. For 10x it's 0.2

In [ ]:
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import pyvips
import re
from IPython.display import display
from ndstorage import Dataset
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import ipywidgets as w

In [ ]:
%matplotlib widget

import matplotlib, matplotlib.pyplot as plt
print("Backend:", matplotlib.get_backend())

In [ ]:
write_directory = Path(ndtiff_folder)

In [ ]:
# Open the dataset
ds = Dataset(ndtiff_folder)

# Convert to numpy array (slow for large datasets; lazy loading is preferred)
images = ds.as_array()

images

In [ ]:
summary = ds.summary_metadata 

stage_positions = summary.get("StagePositions", [])

if len(stage_positions) != len(images):
    raise Exception("Stage positions not the same as number of images. Likely missing some data.")

In [ ]:
num_zlayers = images.shape[3]
num_zlayers

In [ ]:
i = 0  # image index

for zlayer in range(num_zlayers):
    
    arr = images[i][0][0][zlayer]

    # Display
    plt.imshow(arr)
    plt.axis("off")
    plt.title(f"Image {i}, Z-layer {zlayer}")
    plt.show()

In [ ]:
summary = ds.summary_metadata
print(summary.keys())

In [ ]:
with open(write_directory / "summary_metadata.json", "w", encoding="utf-8") as f:
    json.dump(ds.summary_metadata or {}, f, indent=2, ensure_ascii=False, default=str)

In [ ]:
# pull µm/px from AffineTransform; falls back if missing
aff = (ds.summary_metadata.get("UserData", {})
                      .get("AffineTransform", {})
                      .get("scalar", ""))

if aff: 
    ax, bx, by, dy = map(float, aff.split("_"))
    px_x_um = (ax*ax + by*by) ** 0.5
    px_y_um = (bx*bx + dy*dy) ** 0.5
else:
    px_x_um, px_y_um = 0, 0

if given_px_x_um and given_px_y_um: 
    px_x_um = given_px_x_um
    px_y_um = given_px_y_um

print(px_x_um)
print(px_y_um)

In [ ]:
# Only write config.txt if pixel sizes match
if px_x_um == px_y_um:
    config_value = px_x_um

    config_path = os.path.join(write_directory, "config.txt")

    with open(config_path, "w", encoding="utf-8") as f:
        f.write(f"um_per_px={config_value}\n")

    print(f"Wrote config.txt with px_x_um={config_value}")
else:
    print("px_x_um and px_y_um differ; not writing config.txt")

In [ ]:
import json, pandas as pd

summary = ds.summary_metadata  # assumes `ds` already open

# --- concise MDA slice parser: always returns >=1 offset ---
def parse_mda_slices(summary):
    # prefer UserData.MDA_Settings.scalar, else MdaSettings
    mda = (summary.get("UserData") or {}).get("MDA_Settings", {})
    mda = mda.get("scalar") if isinstance(mda, dict) else mda
    if not isinstance(mda, str):
        mda = summary.get("MdaSettings") if isinstance(summary.get("MdaSettings"), str) else None

    offsets, relative, zref = None, True, 0.0
    if mda:
        try:
            jd = json.loads(mda)
            off = jd.get("slices")
            if isinstance(off, (list, tuple)) and len(off) > 0:
                offsets = [float(x) for x in off]
            relative = bool(jd.get("relativeZSlice", True))
            zref     = float(jd.get("zReference", 0.0))
        except Exception:
            pass

    if not offsets:
        S = int(summary.get("Slices") or 1)
        S = max(1, S)
        step = None
        if mda:
            try:
                step = float(json.loads(mda).get("sliceZStepUm"))
            except Exception:
                step = None
        if step and S > 1:
            c = (S - 1) / 2.0
            offsets = [(i - c) * step for i in range(S)]
        else:
            offsets = [0.0] * S
    return offsets, relative, float(zref)

def frame_index(pos_idx: int, z_layer: int, S: int) -> int:
    # position-major, z-minor
    return pos_idx * S + z_layer

# --- Stage positions (one per XY position) ---
stage_positions = summary.get("StagePositions", [])
if not stage_positions:
    raise SystemExit("No StagePositions in summary_metadata — cannot build grid.")
P = len(stage_positions)

# --- Z layer info from metadata ---
slice_offsets_um, relative_slices, zref_um = parse_mda_slices(summary)
S = max(1, len(slice_offsets_um))  # ensure at least 1 layer

# --- Tile size (µm / px) ---
width_px  = summary.get("Width")
height_px = summary.get("Height")
tile_w_um = tile_h_um = None
if width_px and height_px and (px_x_um is not None) and (px_y_um is not None):
    tile_w_um = float(width_px)  * float(px_x_um)
    tile_h_um = float(height_px) * float(px_y_um)

# --- Build records (expand positions across Z) ---
records = []
for p, pos in enumerate(stage_positions):
    label    = pos.get("Label", f"Pos-{p}")
    grid_row = pos.get("GridRow")
    grid_col = pos.get("GridCol")

    x_um = y_um = z_base_um = None
    for dev in pos.get("DevicePositions", []):
        if dev.get("Device") == pos.get("DefaultXYStage", "XYStage"):
            xy = dev.get("Position_um", [None, None])
            if isinstance(xy, (list, tuple)) and len(xy) >= 2:
                x_um, y_um = xy[0], xy[1]
        if dev.get("Device") == pos.get("DefaultZStage", "FocusDrive"):
            zu = dev.get("Position_um", [None])
            z_base_um = zu[0] if isinstance(zu, (list, tuple)) else zu

    for z in range(S):
        dz = float(slice_offsets_um[z])  # safe since len == S
        zb = float(z_base_um) if z_base_um is not None else 0.0
        z_abs_um = zb + dz if relative_slices else dz  # add zref_um if your convention needs it
        records.append({
            "Index":       frame_index(p, z, S),
            "PosIndex":    p,
            "Z_layer":     z,
            "Z_offset_um": dz,
            "Z_base_um":   z_base_um,
            "Z_um":        z_abs_um,
            "Label":       label,
            "GridRow":     grid_row,
            "GridCol":     grid_col,
            "X_um":        x_um,
            "Y_um":        y_um,
        })

df = pd.DataFrame(records).sort_values("Index").reset_index(drop=True)

# --- Relative XY (min-subtracted) ---
if not df[["X_um","Y_um"]].isnull().any().any():
    x0, y0 = float(df["X_um"].min()), float(df["Y_um"].min())
    df["X_rel_um"] = df["X_um"] - x0
    df["Y_rel_um"] = df["Y_um"] - y0
else:
    df["X_rel_um"] = pd.NA
    df["Y_rel_um"] = pd.NA

# --- Relative Z (min-subtracted absolute Z) ---
if not df["Z_um"].isnull().any():
    z0 = float(df["Z_um"].min())
    df["Z_rel_um"] = df["Z_um"] - z0
else:
    df["Z_rel_um"] = pd.NA

# --- Tile & pixel sizes on every row ---
df["TileWidth_um"]  = tile_w_um
df["TileHeight_um"] = tile_h_um
df["TileWidth_px"]  = width_px  if width_px  is not None else pd.NA
df["TileHeight_px"] = height_px if height_px is not None else pd.NA
df["PixelSizeX_um"] = px_x_um   if px_x_um   is not None else pd.NA
df["PixelSizeY_um"] = px_y_um   if px_y_um   is not None else pd.NA

df

In [ ]:
csv_path = write_directory / "summary_table.csv"
df.to_csv(csv_path, index=False)
print("✅ Wrote:", csv_path)

In [ ]:
if px_x_um and px_y_um: 

    # Collect labels + stage coords (µm)
    xy = []
    for i, pos in enumerate(stage_positions):
        try:
            x_um, y_um = pos["DevicePositions"][0]["Position_um"][:2]
            label = pos.get("Label") or f"Pos-{i:04d}"
            xy.append((label, float(x_um), float(y_um)))
        except Exception:
            continue
    
    # Origin at the minimum stage position
    x0 = min(x for _, x, _ in xy)
    y0 = min(y for _, _, y in xy)
    
    IMG_EXT = ".bmp"  # change to ".png" if that’s what you saved
    tileconf = write_directory / "TileConfiguration.txt"
    
    with open(tileconf, "w", encoding="utf-8") as f:
        f.write("dim = 2\n")
        f.writelines(
            f"{label}{IMG_EXT}; ; ({(x - x0)/px_x_um:.3f}, {(y - y0)/px_y_um:.3f})\n"
            for label, x, y in xy
        )
    
    print("✅ Wrote", tileconf)

In [ ]:
df_z0 = df.query("Z_layer == 0").sort_values("Index").reset_index(drop=True)
df_z1 = df.query("Z_layer == 1").sort_values("Index").reset_index(drop=True)
df_z2 = df.query("Z_layer == 2").sort_values("Index").reset_index(drop=True)

#df_z0

In [ ]:
# Corner lookup via GridRow / GridCol
grmin, grmax = df["GridRow"].min(), df["GridRow"].max()
gcmin, gcmax = df["GridCol"].min(), df["GridCol"].max()

In [ ]:
#df.loc[(df["GridRow"]==0) & (df["GridCol"]==0)].iloc[1]

In [ ]:
#df.loc[(df["GridRow"]==grmax) & (df["GridCol"]==gcmax)].iloc[1]

In [ ]:
print(grmax+1)
print(gcmax+2)
print((grmax+1)*(gcmax+1))

In [ ]:
corner_rc = {
    "NW": (grmin, gcmin),
    "NE": (grmin, gcmax),
    "SW": (grmax, gcmin),
    "SE": (grmax, gcmax),
}

# Get one PosIndex per corner (take the first match)
def posindex_at(row, col):
    return int(df.loc[(df["GridRow"]==row) & (df["GridCol"]==col), "PosIndex"].iloc[0])

corner_pos = {name: posindex_at(r, c) for name, (r, c) in corner_rc.items()}

# Already plotted NW; now plot NE, SW, SE
for name in ["NE", "SW", "SE"]:
    i = corner_pos[name]  # PosIndex for this corner
    for zlayer in range(num_zlayers):
        arr = images[i][0][0][zlayer]
        plt.imshow(arr)
        plt.axis("off")
        plt.title(f"{name} corner  |  PosIndex {i}  |  Z-layer {zlayer}")
        plt.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go

# 1) Clean data
cols = ["X_um", "Y_um", "Z_um"]
df_xyz = df.dropna(subset=cols).copy()

# 2) Ranges (min/max define the grid box)
x0, x1 = float(df_xyz["X_um"].min()), float(df_xyz["X_um"].max())
y0, y1 = float(df_xyz["Y_um"].min()), float(df_xyz["Y_um"].max())
z0, z1 = float(df_xyz["Z_um"].min()), float(df_xyz["Z_um"].max())

# 3) Scatter of stage positions
pts = go.Scatter3d(
    x=df_xyz["X_um"],
    y=df_xyz["Y_um"],
    z=df_xyz["Z_um"],
    mode="markers",
    marker=dict(
        size=3,
        opacity=0.9,
        color=df_xyz["Z_um"],          # color by Z for depth cue
        colorscale="Viridis",
        colorbar=dict(title="Z (µm)")
    ),
    hovertemplate=(
        "Index: %{customdata[0]}<br>"
        "Label: %{customdata[1]}<br>"
        "X: %{x:.2f} µm<br>"
        "Y: %{y:.2f} µm<br>"
        "Z: %{z:.2f} µm"
    ),
    customdata=np.stack([
        df_xyz.get("Index", np.arange(len(df_xyz))),
        df_xyz.get("Label", [""]*len(df_xyz))
    ], axis=1)
)

# 4) Bounding box (12 edges)
def box_edges(x0,x1,y0,y1,z0,z1):
    V = [
        (x0,y0,z0),(x1,y0,z0),(x1,y1,z0),(x0,y1,z0),  # bottom (z0)
        (x0,y0,z1),(x1,y0,z1),(x1,y1,z1),(x0,y1,z1)   # top (z1)
    ]
    E = [(0,1),(1,2),(2,3),(3,0), (4,5),(5,6),(6,7),(7,4), (0,4),(1,5),(2,6),(3,7)]
    # concatenate segments with Nones to break lines
    xs, ys, zs = [], [], []
    for a,b in E:
        xs += [V[a][0], V[b][0], None]
        ys += [V[a][1], V[b][1], None]
        zs += [V[a][2], V[b][2], None]
    return go.Scatter3d(
        x=xs, y=ys, z=zs, mode="lines",
        line=dict(color="black", width=2),
        name="Bounds", hoverinfo="skip"
    )

box = box_edges(x0,x1,y0,y1,z0,z1)

# 5) Figure layout (equal axes, nice labels)
fig = go.Figure(data=[pts, box])
fig.update_layout(
    title="Stage Positions (µm)",
    scene=dict(
        xaxis_title="X (µm)",
        yaxis_title="Y (µm)",
        zaxis_title="Z (µm)",
        aspectmode="cube",      # equal scaling on all axes
        xaxis=dict(nticks=6),
        yaxis=dict(nticks=6),
        zaxis=dict(nticks=6),
    ),
    margin=dict(l=0, r=0, t=40, b=0),
    legend=dict(yanchor="top", y=0.98, xanchor="left", x=0.02)
)
fig.show()

In [ ]:
num_firstrectangles = 200
num_lastrectangles  = 200

df_plot = (
    pd.concat([df_z0.head(num_firstrectangles), df_z0.tail(num_lastrectangles)])
      .loc[lambda d: ~d.index.duplicated(keep="first")]
      .copy()
)

#df_plot

In [ ]:
# user controls
num_firstimages = 100
num_lastimages  = 100

df_imgs = (
    pd.concat([df_z0.head(num_firstimages), df_z0.tail(num_lastimages)])
      .loc[lambda d: ~d.index.duplicated(keep="first")]
      .copy()
)

#df_imgs

In [ ]:
px_x_um

In [ ]:
dfv = df_plot

def tile_rgb_from_row(row):
    """Return uint8 RGB tile for the row (H, W, 3). Uses PosIndex/Z_layer if present."""
    if "PosIndex" in row and "Z_layer" in row:
        p = int(row["PosIndex"])
        z = int(row["Z_layer"])
    else:
        # fallback: derive from flattened Index (position-major, z-minor)
        idx = int(row["Index"])
        p, z = divmod(idx, S)

    a = images[p, 0, 0, z]           # (H, W, C) dask -> numpy
    try:
        a = a.compute()
    except Exception:
        pass
    a = np.asarray(a)

    # Normalize to (H, W, 3) uint8
    if a.ndim == 2:
        a = np.repeat(a[..., None], 3, axis=2)
    elif a.shape[-1] == 4:
        a = a[..., :3]
    if a.dtype == np.uint16:
        a = (a >> 8).astype(np.uint8)
    else:
        a = a.astype(np.uint8, copy=False)
    return a

def maybe_downscale(arr, max_side=900):
    """Return arr (H,W,3) possibly downscaled, stays uint8."""
    h, w = arr.shape[:2]
    if max(h, w) <= max_side:
        return arr
    scale = max_side / float(max(h, w))
    new_w = max(1, int(round(w * scale)))
    new_h = max(1, int(round(h * scale)))
    mode = "L" if arr.ndim == 2 else "RGB"
    return np.array(Image.fromarray(arr, mode=mode).resize((new_w, new_h), Image.BILINEAR))

# Vectorized geometry
x0 = dfv["X_rel_um"].to_numpy(float)
y0 = dfv["Y_rel_um"].to_numpy(float)
x1 = (dfv["X_rel_um"] + dfv["TileWidth_um"]).to_numpy(float)
y1 = (dfv["Y_rel_um"] + dfv["TileHeight_um"]).to_numpy(float)
xc = (x0 + x1) / 2
yc = (y0 + y1) / 2
labels = dfv["Label"].astype(str).to_list()

# One-shot shapes + annotations
shapes = [
    dict(type="rect", x0=a, y0=b, x1=c, y1=d, line=dict(width=1, color="black"))
    for a, b, c, d in zip(x0, y0, x1, y1)
]
ann = [
    dict(x=a, y=b, text=t, showarrow=False, font=dict(size=10))
    for a, b, t in zip(xc, yc, labels)
]

fig = go.Figure()

# Invisible trace to set axis limits automatically
fig.add_trace(
    go.Scatter(
        x=[x0.min(), x1.max()],
        y=[y0.min(), y1.max()],
        mode="markers",
        opacity=0,
        showlegend=False,
        hoverinfo="skip",
    )
)

# Minimal layout edits (no per-item updates)
fig.update_layout(
    title="Tiles (0,0 at top-left)",
    shapes=shapes,
    annotations=ann,
    dragmode="zoom",
    height=750,
    margin=dict(l=60, r=20, t=50, b=60),
    plot_bgcolor="white",
)
fig.update_xaxes(title="X (µm)", showgrid=False, ticks="outside",
                 showline=True, mirror=True, zeroline=False)
fig.update_yaxes(title="Y (µm)", showgrid=False, ticks="outside",
                 showline=True, mirror=True, zeroline=False,
                 scaleanchor="x", autorange="reversed")

# --- then add images only for first/last selections ---
for _, r in df_imgs.iterrows():
    arr8 = tile_rgb_from_row(r)                 # <-- robust loader
    arr8 = maybe_downscale(arr8, max_side=900)

    fig.add_layout_image(
        dict(
            source=Image.fromarray(arr8, mode="RGB"),
            x=float(r["X_rel_um"]),
            y=float(r["Y_rel_um"]),
            xref="x", yref="y",
            sizex=float(r["TileWidth_um"]),
            sizey=float(r["TileHeight_um"]),
            xanchor="left", yanchor="top",
            sizing="stretch",
            layer="below",
            opacity=1.0
        )
    )

fig.show()

In [ ]:
# Build a downsampled mosaic of the first N tiles.
# - Keeps the extent in ORIGINAL microns (so clicks report true µm)
# - Downsamples on the fly (resizes each tile before pasting)
#
# Controls:
#   * max_canvas_px: limit on the longer side of the mosaic canvas (pixels)
#   * or px_per_um_target: desired display resolution (pixels per µm)
#      (If both given, the tighter limit is used.)
#
# Returns:
#   merged_rgb   : uint8 HxWx3
#   extent_um    : [xmin, xmax, ymin, ymax] in µm  (use for imshow(..., extent=..., origin='upper'))
#   scale        : downsample factor (0<scale<=1) applied to pixel sizes
#   px_per_um    : original px/µm of the tiles (pre-downsample)
#   px_per_um_eff: effective px/µm in the merged image (post-downsample) = px_per_um * scale

import numpy as np
from PIL import Image
import os

def merge_first_tiles_downsampled(
    num_firstimages: int,
    df_imgs,
    tile_loader,
    max_canvas_px: int = 4000,
    px_per_um_target: float | None = None
):
    subset = df_imgs.head(num_firstimages)
    if subset.empty:
        raise ValueError("df_imgs is empty or num_firstimages=0.")

    # infer original px/µm from first tile
    r0 = subset.iloc[0]
    a0 = tile_loader(r0)
    if a0.ndim == 2: a0 = np.repeat(a0[..., None], 3, axis=2)
    if a0.dtype != np.uint8: a0 = a0.astype(np.uint8, copy=False)
    px_per_um = a0.shape[1] / float(r0["TileWidth_um"])

    # mosaic bounds in µm
    xmin = float(subset["X_rel_um"].min())
    ymin = float(subset["Y_rel_um"].min())
    xmax = float((subset["X_rel_um"] + subset["TileWidth_um"]).max())
    ymax = float((subset["Y_rel_um"] + subset["TileHeight_um"]).max())
    width_um  = xmax - xmin
    height_um = ymax - ymin

    # original full-res canvas size (pixels)
    W_full = width_um  * px_per_um
    H_full = height_um * px_per_um

    # choose downsample scale
    scale_limits = []
    if max_canvas_px is not None:
        scale_limits.append(min(1.0, max_canvas_px / max(W_full, H_full)))
    if px_per_um_target is not None:
        scale_limits.append(min(1.0, px_per_um_target / px_per_um))
    scale = min(scale_limits) if scale_limits else 1.0
    if scale <= 0:
        raise ValueError("Computed non-positive scale; check inputs.")

    # canvas size after downsample
    W = int(np.ceil(W_full * scale))
    H = int(np.ceil(H_full * scale))
    canvas = Image.new("RGB", (W, H), (255, 255, 255))

    for _, r in subset.iterrows():
        a = tile_loader(r)
        if a.ndim == 2: a = np.repeat(a[..., None], 3, axis=2)
        if a.dtype != np.uint8: a = a.astype(np.uint8, copy=False)
        # resize tile if we’re downsampling
        if scale != 1.0:
            new_w = max(1, int(round(a.shape[1] * scale)))
            new_h = max(1, int(round(a.shape[0] * scale)))
            a = np.array(Image.fromarray(a, mode="RGB").resize((new_w, new_h), Image.BILINEAR))
        img = Image.fromarray(a, mode="RGB")

        # paste position (top-left origin), downsampled offset
        x_px = int(round((float(r["X_rel_um"]) - xmin) * px_per_um * scale))
        y_px = int(round((float(r["Y_rel_um"]) - ymin) * px_per_um * scale))
        canvas.paste(img, (x_px, y_px))

    merged_rgb = np.asarray(canvas)
    extent_um = [xmin, xmax, ymin, ymax]
    px_per_um_eff = px_per_um * scale

    outname = f"merged_first{num_firstimages}.png"
    outpath = os.path.join(write_directory, outname)
    canvas.save(outpath)

    print("Merged image saved to:", os.path.abspath(outpath))
    
    print(f"Saved downsampled mosaic to: {os.path.abspath(outpath)}")
    print(f"Canvas: {merged_rgb.shape[1]}x{merged_rgb.shape[0]} px | scale={scale:.4f} | px/µm eff={px_per_um_eff:.4f}")

    return merged_rgb, extent_um, scale, px_per_um, px_per_um_eff

# ---- run it (example) ----
num_firstimages = 40             # how many tiles
max_canvas_px   = 5000           # limit longest side to ~5k px
px_per_um_target = None          # or e.g. 0.2 px/µm if you prefer a target density
merged_rgb, extent, scale, px_per_um, px_per_um_eff = merge_first_tiles_downsampled(
    num_firstimages, df_imgs, tile_rgb_from_row,
    max_canvas_px=max_canvas_px,
    px_per_um_target=px_per_um_target
)

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
#import ipywidgets as w
from IPython.display import Javascript, display

# ---------- figure ----------
fig, ax = plt.subplots(figsize=(9, 4))
ax.set_title("Zoom (toolbar) to inspect, then LEFT-click two points → Process")
im = ax.imshow(merged_rgb, extent=extent, origin='upper')
ax.set_xlabel("X (µm)")
ax.set_ylabel("Y (µm)")
ax.set_aspect('equal')

# visuals for picks
p1_artist, = ax.plot([], [], 'o', ms=8, label="P1")
p2_artist, = ax.plot([], [], 'o', ms=8, label="P2")
line_artist, = ax.plot([], [], '-', lw=2, label="P1–P2")
ax.legend(loc="upper right")

picked = []   # holds up to 2 tuples (x_um, y_um)

# ---------- UI ----------
btn_reset   = w.Button(description="Reset",   button_style="warning")
btn_process = w.Button(description="Process", button_style="success")
status_out  = w.Output(layout={"border":"1px solid #ddd"})
display(w.HBox([btn_reset, btn_process]), status_out)

outname = f"rotation.txt"
outpath = os.path.join(write_directory, outname)

def _status(msg):
    status_out.clear_output(wait=True)
    with status_out:
        print(msg)

def reset(_btn=None):
    picked.clear()
    p1_artist.set_data([], [])
    p2_artist.set_data([], [])
    line_artist.set_data([], [])
    fig.canvas.draw_idle()
    _status("Cleared picks. Left-click two points, then press Process.")
btn_reset.on_click(reset)

def rotation_to_horizontal_deg(angle_deg: float) -> float:
    """
    Given the absolute segment angle (deg, CCW +X), return the smallest CCW rotation
    in [-90, +90] deg that makes the segment horizontal (angle ≡ 0 mod 180).
    """
    # Wrap angle to [-180, 180)
    a = (angle_deg + 180) % 360 - 180
    rot = -a                      # rotate so angle→0
    if rot > 90:  rot -= 180
    if rot <= -90: rot += 180
    return rot

def process(_btn=None):
    if len(picked) != 2:
        _status("Need exactly two points. Left-click twice, then press Process.")
        return

    (x1, y1), (x2, y2) = picked
    dx, dy = x2 - x1, y2 - y1
    dist = float(np.hypot(dx, dy))
    if dist == 0:
        _status("The two points coincide. Pick two distinct points.")
        return

    # Angle of segment; origin='upper' means Y grows downward, but atan2(dy,dx) is correct.
    angle_deg = float(np.degrees(np.arctan2(dy, dx)))
    rot_deg = float(rotation_to_horizontal_deg(angle_deg))

    # Write file
    with open(outpath, "w", encoding="utf-8") as f:
        f.write(f"{rot_deg:.6f}\n")

    config_path = out_dir / "config.txt"
    with open(config_path, "a", encoding="utf-8") as f:
        f.write(f"rotation={rot_deg:.6f}\n")

    # Visual + status
    _status(
        f"P1=({x1:.2f}, {y1:.2f}) µm | P2=({x2:.2f}, {y2:.2f}) µm\n"
        f"Δ=({dx:.2f}, {dy:.2f}) µm  | |Δ|={dist:.2f} µm  | angle={angle_deg:.2f}°\n"
        f"→ rotate by {rot_deg:.2f}° (CCW) to make the line horizontal\n"
        f"Wrote: {outpath}"
    )

    # optional: copy to clipboard
    payload = {"P1_um":[x1,y1], "P2_um":[x2,y2], "angle_deg":angle_deg, "rotation_deg":rot_deg}
    try:
        display(Javascript(f"navigator.clipboard.writeText({repr(payload)})"))
    except Exception:
        pass

btn_process.on_click(process)

def onclick(event):
    # Only inside axes, left button, and not while a toolbar tool is active
    if event.inaxes is not ax:
        return
    if getattr(event, "button", 1) != 1:  # 1=left, 2=middle, 3=right
        return
    tb = getattr(fig.canvas, "toolbar", None)
    if tb and getattr(tb, "mode", ""):
        return

    x, y = float(event.xdata), float(event.ydata)
    if len(picked) == 0:
        picked[:] = [(x, y)]
        p1_artist.set_data([x], [y])
        p2_artist.set_data([], [])
        line_artist.set_data([], [])
        _status(f"Picked P1=({x:.2f}, {y:.2f}) µm. Pick P2, then press Process.")
    elif len(picked) == 1:
        picked.append((x, y))
        p2_artist.set_data([x], [y])
        line_artist.set_data([picked[0][0], x], [picked[0][1], y])
        _status("P1 and P2 selected. Press Process.")
    else:
        picked[:] = [(x, y)]
        p1_artist.set_data([x], [y])
        p2_artist.set_data([], [])
        line_artist.set_data([], [])
        _status(f"Restarted: P1=({x:.2f}, {y:.2f}) µm. Pick P2, then press Process.")

    fig.canvas.draw_idle()

cid = fig.canvas.mpl_connect('button_press_event', onclick)
fig

## If everything is ok?

If so then click on the button below. It will:
- Create ready_to_process.txt (trigger file for processing on nucleonics)
- Move the files into the micro folder

In [ ]:

# ----------------------------------------------------
# CONFIGURATION
# ----------------------------------------------------
# e.g. write_directory = r"C:\Users\MITLENR\leica-images\uchikoshi01_pd-fo-05\_1" at the top of the notebook

# Base destination for sample folders
base_new_dir = r"C:\Users\MITLENR\micro"

# We want to end up with this structure:
# C:\Users\MITLENR\micro\uchikoshi01_pd-fo-05\_1

# Extract parent e.g ("uchikoshi01_pd-fo-05") and child ("_1")
parent = os.path.basename(os.path.dirname(write_directory))
child  = os.path.basename(write_directory)

# Destination parent folder:
new_directory_parent = os.path.join(base_new_dir, parent)

# Final destination folder:
new_directory = os.path.join(new_directory_parent, child)


# ----------------------------------------------------
# UI elements
# ----------------------------------------------------
btn_ok = w.Button(
    description="OK",
    button_style="success"
)
status = w.Output(layout={"border": "1px solid #ddd"})

display(status, btn_ok)

with status:
    print("Are you ready? Click OK to continue.")


# ----------------------------------------------------
# Button callback
# ----------------------------------------------------
def on_ok_clicked(_):
    status.clear_output(wait=True)
    with status:
        print("Button clicked. Creating ready_to_process.txt...")

    # ----------------------------------------------------
    # 1. Create ready_to_process.txt inside the source folder
    # ----------------------------------------------------
    ready_file = os.path.join(write_directory, "ready_to_process.txt")
    with open(ready_file, "w", encoding="utf-8"):
        pass

    with status:
        print(f"Created: {ready_file}")

    # ----------------------------------------------------
    # 2. Create destination parent folder
    # ----------------------------------------------------
    os.makedirs(new_directory_parent, exist_ok=True)

    with status:
        print(f"Destination parent ready: {new_directory_parent}")
        print("Now moving the entire folder...")

    # ----------------------------------------------------
    # 3. Move the entire folder in one operation
    # ----------------------------------------------------
    shutil.move(write_directory, new_directory)

    with status:
        print("Move complete.")
        print(f"Folder moved to: {new_directory}")


# Connect button handler
btn_ok.on_click(on_ok_clicked)
